In [ ]:
import numpy as np

class MatrixOperations:
    def __init__(self, matrix):
        if not isinstance(matrix, np.ndarray):
            matrix = np.array(matrix)
        self.matrix = matrix
        self.m, self.n = matrix.shape

    def rref(self, B: np.ndarray = None):

        if B is None:
            augmented_matrix = self.matrix.copy()
        else:
            if not isinstance(B, np.ndarray):
                B = np.array(B)

            if len(np.shape(B)) == 1:
                B = B.reshape(-1, 1)

            assert self.matrix.shape[0] == B.shape[0], \
                "Incompatible dimensions - right matrix must have the same number of rows as the left matrix"
            
            augmented_matrix = np.hstack((self.matrix.copy(), B.copy()))

        augmented_matrix = augmented_matrix.astype(float)
        m,n = self.m, self.n

        # Loop through each column
        l = 0 # negative row offset
        for i in range(n):

            # Skip columns that are all zero
            if np.allclose(augmented_matrix[:, i], 0):
                l += 1
                continue

            # Find a non-zero pivot and swap rows if necessary
            j = i - l
            while j < m: # Loop through each element in the row
                if augmented_matrix[j, i] != 0:
                    if j != i - l:
                        augmented_matrix[[i - l, j]] = augmented_matrix[[j, i - l]]
                    break

                j += 1
                
            # Catches case where n > m and up to m pivots have been found
            if j == m:
                continue
            
            augmented_matrix[i-l] = augmented_matrix[i-l] / augmented_matrix[i-l, i]

            # Eliminate the other rows
            for k in range(m):
                if k != i:
                    augmented_matrix[k] -= augmented_matrix[i] * augmented_matrix[k, i]

        # Return the inverted matrix
        return augmented_matrix

    def inverse(self):
        # Check if the matrix is square
        if self.matrix.shape[0] != self.matrix.shape[1]:
            print('Matrix must be square to compute its inverse')
            return None

        # Create an identity matrix of the same size
        n = self.matrix.shape[0]
        identity_matrix = np.eye(n)

        augmented_matrix = self.rref(B=identity_matrix)

        # Check if the left side is now the identity matrix
        if not np.allclose(augmented_matrix[:, :n], np.eye(n)):
            print('Matrix cannot be inverted - not all rows are independent')
            return None

        return augmented_matrix[:, n:]
    
    def solve(self, b, granularity=2, debug=False):
        augmented_matrix = self.rref(B=b)

        # Check if all 0 rows in left matrix correspond to 0 in right matrix
        for i in range(self.m):
            if np.allclose(augmented_matrix[i, :-1], 0) and not np.isclose(augmented_matrix[i, -1], 0):
                print('No solutions exist')
                return None
            
        # Loop through each column and check if a column is all zero
        dont_care = []
        for j in range(self.n):
            if np.allclose(augmented_matrix[:, j], 0):
                dont_care.append(j+1)

        # Check each if there all zero column - set this as free variable
        # Loop through each row 
            # Find the leading 1 in the row
            # Suceeding columns are subtracted from the right hand side
            # If no leading 1, then free variable
        
        pre_final_solution = augmented_matrix[:, -1].copy()
        free_variables = []
        
        for i in range(self.m):
            j = 0 # could be improved with negative offset but my brain is fried

            # Find the leading 1 in the row
            while j < self.n:
                if np.allclose(augmented_matrix[i, j], 1):
                    j2 = j + 1
                    solution_str = f'x{j+1} = {pre_final_solution[i]}'
                    while j2 < self.n:
                        if not np.isclose(augmented_matrix[i, j2], 0):
                            free_variables.append(j2+1)
                            if augmented_matrix[i, j2] > 0:
                                solution_str += f' - ({augmented_matrix[i, j2]:.{granularity}f})x{j2+1}'
                            else:
                                solution_str += f' + ({-augmented_matrix[i, j2]:.{granularity}f})x{j2+1}'
                        j2 += 1
                        
                    j += 1
                    break

                j += 1

            if i+1 in free_variables:
                print(f'x{i+1} is a free variable')
            elif i+1 in dont_care:
                print(f'x{i+1} is a "dont care" variable')
            else:
                print(f'{solution_str}')

        if debug:
            return augmented_matrix

        if len(free_variables) > 0 or len(dont_care) > 0:
            return 'Check the printed solution for details'
        else:
            return augmented_matrix[:, -1]

    def svd(self):
        pass

    def pinv(self):
        passmatri


In [378]:
A = [
    [1,0,0],
    [0,0,1],
    [0,0,0]
]

b = [1,2,3]

matrix_ops = MatrixOperations(A)
solution = matrix_ops.solve(b)
print("Solution for Ax = b:\n", solution)

No solutions exist
Solution for Ax = b:
 None


In [380]:
matrix_ops = MatrixOperations(a)
inverse_a = matrix_ops.inverse()
print("Original Matrix:\n", a)
print("Inverse Matrix:\n", inverse_a)

Original Matrix:
 [[1 0 1]
 [0 7 6]
 [3 0 2]]
Inverse Matrix:
 [[-2.          0.          1.        ]
 [-2.57142857  0.14285714  0.85714286]
 [ 3.         -0.         -1.        ]]


In [131]:
a @ inverse_a  # Should be close to identity matrix

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [381]:
import numpy as np

A = np.array([
    [1, 1, 5],
    [2, 2, 6],
    [3, 3, 7]
], dtype=float)

b = np.array([6, 8, 10], dtype=float)

# Since A may not be full rank, use lstsq instead of inv
x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

print("Solution x:", x)
print("Residuals:", residuals)
print("Rank of A:", rank)
print("Singular values:", s)

Solution x: [0.5 0.5 1. ]
Residuals: []
Rank of A: 2
Singular values: [1.16873598e+01 1.18558911e+00 2.90794310e-16]
